# Objetivo

Entender as relações entre as variáveis explicativas e as explicativas e a resposta. A ideia inicial é utilizar o pagamento como variável resposta.

# Pacotes e funções

In [29]:
import pandas as pd
from scipy.stats import chi2_contingency

In [2]:
%run ../99.Funcoes_auxiliares/Funcoes.py

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Leitura da base de treino

In [4]:
dados_full = pd.read_parquet('../0.Avalia_Dados/Dados/treino.parquet')

In [5]:
# Colunas desconsideradas: id, documento, mes_pagamento, divida_restante, per_pago, genero
# Colunas desconsideradas momentaneamente para o problema de regressão: target

colunas_remove = ['id', 'documento', 'mes_pagamento', 'divida_restante', 'per_pago', 'GENERO', 'target']

dados_full.drop(columns=colunas_remove, inplace=True)

# Removendo duplicatas, se houver
dados_full.drop_duplicates(inplace=True)

In [5]:
dados_full.shape

(96109, 249)

In [22]:
dados_full.head(3)

,dias_atraso,segmento_veiculo,saldo_vencido,pagamento,IDADE,UF,VAR_1,VAR_2,VAR_3,VAR_4,VAR_5,VAR_7,VAR_8,VAR_9,VAR_11,VAR_12,VAR_15,VAR_17,VAR_18,VAR_21,VAR_22,VAR_23,VAR_25,VAR_32,VAR_33,VAR_34,VAR_35,VAR_40,VAR_42,VAR_43,VAR_44,VAR_45,VAR_46,VAR_47,VAR_48,VAR_49,VAR_50,VAR_51,VAR_52,VAR_53,VAR_54,VAR_55,VAR_56,VAR_57,VAR_58,VAR_59,VAR_60,VAR_61,VAR_62,VAR_63,VAR_64,VAR_65,VAR_66,VAR_67,VAR_68,VAR_69,VAR_70,VAR_71,VAR_72,VAR_73,VAR_74,VAR_75,VAR_76,VAR_77,VAR_78,VAR_79,VAR_80,VAR_81,VAR_82,VAR_83,VAR_84,VAR_85,VAR_86,VAR_87,VAR_88,VAR_89,VAR_90,VAR_91,VAR_92,VAR_93,VAR_94,VAR_95,VAR_96,VAR_97,VAR_98,VAR_99,VAR_100,VAR_101,VAR_102,VAR_103,VAR_104,VAR_105,VAR_106,VAR_107,VAR_108,VAR_109,VAR_110,VAR_111,VAR_112,VAR_113,VAR_114,VAR_115,VAR_116,VAR_117,VAR_118,VAR_119,VAR_120,VAR_121,VAR_122,VAR_123,VAR_124,VAR_125,VAR_126,VAR_127,VAR_128,VAR_129,VAR_130,VAR_131,VAR_132,VAR_133,VAR_134,VAR_135,VAR_138,VAR_139,VAR_140,VAR_141,VAR_142,VAR_143,VAR_144,VAR_145,VAR_146,VAR_147,VAR_148,VAR_149,VAR_150,VAR_151,VAR_152,VAR_153,VAR_154,VAR_155,VAR_156,VAR_157,VAR_158,VAR_159,VAR_160,VAR_161,VAR_162,VAR_163,VAR_164,VAR_165,VAR_166,VAR_167,VAR_168,VAR_169,VAR_170,VAR_171,VAR_172,VAR_173,VAR_174,VAR_175,VAR_176,VAR_177,VAR_178,VAR_179,VAR_180,VAR_181,VAR_182,VAR_183,VAR_184,VAR_185,VAR_186,VAR_187,VAR_188,VAR_189,VAR_190,VAR_191,VAR_192,VAR_193,VAR_194,VAR_195,VAR_196,VAR_197,VAR_198,VAR_199,VAR_200,VAR_201,VAR_202,VAR_203,VAR_204,VAR_205,VAR_206,VAR_207,VAR_208,VAR_209,VAR_210,VAR_211,VAR_212,VAR_213,VAR_214,VAR_215,VAR_216,VAR_217,VAR_218,VAR_219,VAR_220,VAR_221,VAR_222,VAR_223,VAR_224,VAR_225,VAR_226,VAR_227,VAR_228,VAR_229,VAR_230,VAR_231,VAR_232,VAR_233,VAR_238,VAR_240,VAR_250,VAR_253,VAR_254,VAR_256,VAR_257,VAR_258,VAR_259,VAR_260,VAR_261,VAR_262,VAR_263,VAR_264,VAR_265,VAR_267,VAR_268,VAR_269,VAR_271,VAR_304,VAR_305,VAR_306,VAR_307,VAR_308,VAR_309,VAR_310,VAR_311,VAR_312,VAR_313,VAR_314,VAR_315
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
257349,87,leves,1966.20,0.0,22.721,None,None,None,None,NaN,0.112,NaN,NaN,1.583333,NaN,NaN,NaN,NaN,NaN,0.000,0.133,0.038,None,NaN,NaN,NaN,NaN,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1522.090503,E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.503700,34.207269,0.0,2.0,0.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,N,None,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,None,NaN,F,1.0
257347,80,leves,6177.81,0.0,42.504,UF_7,D,E,MEDIA,1.0,0.744,0.208,0.205,0.500000,1.837035,3.039882,0.124,0.054,0.0,0.159,0.039,0.000,IMPOSTO A PAGAR,0.434656,0.560532,0.477024,NaN,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,ALTISSIMA,ALTISSIMA,ALTISSIMA,ALTA,ALTISSIMA,ALTA,ALTISSIMA,ALTISSIMA,ALTISSIMA,ALTISSIMA,ALTA,ALTA,ALTISSIMA,ALTISSIMA,ALTA,ALTA,ALTISSIMA,ALTISSIMA,ALTA,ALTISSIMA,ALTISSIMA,ALTISSIMA,ALTA,ALTISSIMA,BAIXISSIMA,ALTA,ALTISSIMA,ALTA,ALTISSIMA,ALTA,ALTISSIMA,ALTA,ALTISSIMA,ALTA,BAIXISSIMA,ALTISSIMA,MEDIO,PROXIMO,MEDIO,PROXIMO,PROXIMO,MEDIO,PROXIMO,MEDIO,MEDIO,LONGE,LONGE,MEDIO,MEDIO,MEDIO,LONGE,PROXIMO,PROXIMO,

In [ ]:
# Somente as variáveis categóricas

dados_full.head().select_dtypes(include=['object']).columns

(5, 112)

In [23]:
dados_full.head(3).select_dtypes(include=['object'])

,segmento_veiculo,UF,VAR_1,VAR_2,VAR_3,VAR_25,VAR_42,VAR_43,VAR_44,VAR_45,VAR_46,VAR_47,VAR_48,VAR_49,VAR_50,VAR_51,VAR_52,VAR_53,VAR_54,VAR_55,VAR_56,VAR_57,VAR_58,VAR_59,VAR_60,VAR_61,VAR_62,VAR_63,VAR_64,VAR_65,VAR_66,VAR_67,VAR_68,VAR_69,VAR_70,VAR_71,VAR_72,VAR_73,VAR_74,VAR_75,VAR_76,VAR_77,VAR_78,VAR_79,VAR_80,VAR_81,VAR_82,VAR_83,VAR_84,VAR_85,VAR_86,VAR_87,VAR_88,VAR_89,VAR_90,VAR_91,VAR_92,VAR_93,VAR_94,VAR_95,VAR_96,VAR_97,VAR_98,VAR_99,VAR_100,VAR_101,VAR_102,VAR_103,VAR_104,VAR_105,VAR_106,VAR_107,VAR_108,VAR_109,VAR_110,VAR_111,VAR_112,VAR_113,VAR_114,VAR_115,VAR_116,VAR_117,VAR_118,VAR_119,VAR_120,VAR_121,VAR_122,VAR_123,VAR_124,VAR_125,VAR_126,VAR_127,VAR_128,VAR_129,VAR_130,VAR_131,VAR_132,VAR_133,VAR_135,VAR_233,VAR_238,VAR_240,VAR_250,VAR_256,VAR_257,VAR_304,VAR_306,VAR_307,VAR_308,VAR_311,VAR_312,VAR_314
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
257349,leves,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,E,None,None,None,None,N,None,None,None,None,None,None,None,F
257347,leves,UF_7,D,E,MEDIA,IMPOSTO A PAGAR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,ALTISSIMA,ALTISSIMA,ALTISSIMA,ALTA,ALTISSIMA,ALTA,ALTISSIMA,ALTISSIMA,ALTISSIMA,ALTISSIMA,ALTA,ALTA,ALTISSIMA,ALTISSIMA,ALTA,ALTA,ALTISSIMA,ALTISSIMA,ALTA,ALTISSIMA,ALTISSIMA,ALTISSIMA,ALTA,ALTISSIMA,BAIXISSIMA,ALTA,ALTISSIMA,ALTA,ALTISSIMA,ALTA,ALTISSIMA,ALTA,ALTISSIMA,ALTA,BAIXISSIMA,ALTISSIMA,MEDIO,PROXIMO,MEDIO,PROXIMO,PROXIMO,MEDIO,PROXIMO,MEDIO,MEDIO,LONGE,LONGE,MEDIO,MEDIO,MEDIO,LONGE,PROXIMO,PROXIMO,LONGE,MUITO LONGE,MEDIO,PROXIMO,MEDIO,MEDIO,MEDIO,PROXIMO,PROXIMO,MEDIO,MEDIO,MEDIO,MUITO PROXIMO,MEDIO,MEDIO,MEDIO,LONGE,MEDIO,MEDIO,MEDIO,MEDIO,MEDIO,D,S,N,N,S,N,N,PROXIMO,PROXIMO,LONGE,PROXIMO,URBANO,SUL,A
257375,leves,UF_12,None,E,ALTISSIMA,None,S,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXISSIMA,BAIXA,BAIXISSIMA,BAIXISSIMA,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO PROXIMO,MUITO LONGE,MUITO LONGE,LONGE,MUITO LONGE,LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,LONGE,MUITO LONGE,MUITO LONGE,LONGE,LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,MUITO PROXIMO,LONGE,LONGE,MUITO LONGE,MUITO LONGE,MUITO LONGE,E,None,None,None,None,N,None,MUITO LONGE,PROXIMO,LONGE,MEDIO,RURAL,NORDESTE,F


In [ ]:
# Tabelas de frequências das variáveis categóricas
    # Variáveis categóricas com mais de 85% dos valores concentrados em um nível serão desconsideradas: 
    # VAR_2, VAR_42, VAR_44, VAR_45, VAR_46, VAR_47, VAR_48, VAR_50, VAR_51, VAR_52, VAR_53, VAR_55, VAR_56, VAR_57, VAR_113, VAR_256

for i in dados_full.select_dtypes(include=['object']).columns:
    print('===================================')
    print(i)
    print(conta_categorias(variavel=i, df = dados_full)[1])

segmento_veiculo
  segmento_veiculo      count
0            leves  70.617736
1            motos  29.382264
UF
       UF      count
0    UF_0  33.166509
1    UF_7   7.783870
2    UF_8   7.270911
3   UF_15   5.542665
4   UF_21   5.043232
5    UF_5   4.190034
6    None   3.599039
7   UF_13   3.259840
8    UF_1   3.234869
9   UF_12   3.118334
10  UF_17   3.062148
11   UF_9   2.772893
12  UF_16   2.750003
13   UF_3   2.521096
14  UF_14   2.472193
15   UF_4   1.731368
16   UF_6   1.694951
17  UF_11   1.096671
18  UF_23   0.936437
19   UF_2   0.893777
20  UF_19   0.767878
21  UF_24   0.639898
22  UF_22   0.552498
23  UF_20   0.521283
24  UF_25   0.506716
25  UF_18   0.417235
26  UF_10   0.353765
27  UF_26   0.099887
VAR_1
  VAR_1      count
0     E  55.246647
1  None  23.004089
2     D  16.578052
3     C   4.527151
4     B   0.456773
5     A   0.187287
VAR_2
  VAR_2      count
0     E  94.274209
1  None   3.543893
2     C   1.114360
3     D   0.987421
4     B   0.071793
5     A   0.008324
VAR

In [26]:
tc = pd.crosstab(dados_full['VAR_54'], dados_full['VAR_58'], margins=True, margins_name="subtotal")
tc

VAR_58,N,S,subtotal
VAR_54,,,
N,65819,12667,78486
S,8329,4558,12887
subtotal,74148,17225,91373


In [ ]:
# https://www.analyticsvidhya.com/blog/2023/03/discovering-insights-with-chi-square-tests-a-hands-on-approach-in-python/
# https://www.geeksforgeeks.org/python-pearsons-chi-square-test/

chi, p, dof, expected = chi2_contingency(tc)

In [31]:
p

np.float64(0.0)

In [ ]:
# Somente as variáveis numéricas

dados_full.head().select_dtypes(exclude=['object']).shape

(5, 137)